# Example: Create a cell type reference gene expression file from a scRNA-seq atlas with cell type annotation. 
- Using Human colon cancer atlas as example.
- Data downloaded from GSE178341, with h5 and cell type labels. 
## We already know cutoffs for good quality so Step0 can be skipped:
- MIN_GENES = 100
- MIN_COUNTS = 100
- MIN_CELLS = 3
- MAX_MITO_PERCENT=27.77

In [13]:
import os
import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import sys
# add STHD path
sys.path.append('../')

# (pip version to be published)
# pip install STHD

# Step 0 (Optional): Find Good QC cutoffs  
## Load scRNA reference

In [2]:
crc_dir = '../testdata/crc_scrna/'
adata = sc.read_10x_h5('../testdata/crc_scrna/GSE178341_crc10x_full_c295v4_submit.h5')

/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


## Load cell type names and add to adata

In [3]:
def load_global_celltype(crc_dir):
    '''
    For colon cancer atlas
    '''
    cell_type_tsne_type_file = 'crc10x_tSNE_cl_global.tsv'
    tmp = pd.read_table( os.path.join(crc_dir, cell_type_tsne_type_file), skiprows=1, index_col = 0) # same cell type and number
    return(tmp)

celltypes = load_global_celltype(crc_dir)
# qc
celltypes = celltypes.loc[ celltypes.index.intersection(adata.obs.index) ]
celltypes_group1 = celltypes.groupby('group.1').count().index.tolist()
celltypes_group = celltypes.groupby('group').count().index.tolist()

adata.obsm['X_tsne'] = celltypes[['numeric','numeric.1']].loc[adata.obs.index].values
adata.obs['group']=celltypes['group'].loc[adata.obs.index].values
adata.obs['group.1']=celltypes['group.1'].loc[adata.obs.index].values

## Optional plotting

In [ ]:
sc.pl.tsne(adata,color='group.1')
sc.pl.tsne(adata,color='group')

## Finding QC criterion: Optional for first time QC: QC and mitochondrial control. can skip if you know the QC cutoffs.

In [7]:

MIN_GENES = 100
MIN_COUNTS = 100
MIN_CELLS = 3
random_state = 42


In [ ]:
adata.var_names_make_unique()
adata.var['mt'] = adata.var_names.str.startswith('MT-') | adata.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

In [ ]:
import matplotlib.pyplot as plt
sc.pl.tsne(adata,color='pct_counts_mt')
#plt.savefig('../analysis/figure/crc95_ref_qc_pct_counts_mt.pdf', dpi=300)

In [ ]:
import seaborn as sns
sns.displot(adata.obs['pct_counts_mt'].values)
plt.axvline(27.7, color='pink')
plt.xlabel('pct_counts_mt')

In [11]:
# either

MAX_MITO_PERCENT = adata.obs['pct_counts_mt'].mean() + adata.obs['pct_counts_mt'].std()
print(f'Mito cutoff option: {MAX_MITO_PERCENT}')

Mito cutoff option: 27.771158613939527


In [12]:
sc.pp.filter_cells(adata, min_genes = MIN_GENES)  
sc.pp.filter_cells(adata, min_counts = MIN_COUNTS )   
sc.pp.filter_genes(adata, min_cells = MIN_CELLS)

print('pre mt filtering ', adata.shape[0])
adata = adata[adata.obs.pct_counts_mt < MAX_MITO_PERCENT , :]
print('post mt filtering ', adata.shape[0])


pre mt filtering  370115
post mt filtering  303358


# Step 1: Read in the reference scRNA and QC 

In [14]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
from tqdm import tqdm

In [5]:
crc_dir = '../testdata/crc_scrna/'
adata = sc.read_10x_h5('../testdata/crc_scrna/GSE178341_crc10x_full_c295v4_submit.h5')

/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


## Get cell type and load to raw data

In [6]:
def load_global_celltype(crc_dir):
    '''
    For colon cancer atlas
    '''
    cell_type_tsne_type_file = 'crc10x_tSNE_cl_global.tsv'
    tmp = pd.read_table( os.path.join(crc_dir, cell_type_tsne_type_file), skiprows=1, index_col = 0) # same cell type and number
    return(tmp)

celltypes = load_global_celltype(crc_dir)
# qc
celltypes = celltypes.loc[ celltypes.index.intersection(adata.obs.index) ]
celltypes_group1 = celltypes.groupby('group.1').count().index.tolist()
celltypes_group = celltypes.groupby('group').count().index.tolist()

In [7]:
adata.obsm['X_tsne'] = celltypes[['numeric','numeric.1']].loc[adata.obs.index].values
adata.obs['group']=celltypes['group'].loc[adata.obs.index].values
adata.obs['group.1']=celltypes['group.1'].loc[adata.obs.index].values

## KEEP the raw counts

In [8]:
adata.var_names_make_unique()
if('counts') not in adata.layers.keys():
    adata.layers["counts"] = adata.X.copy()


/hpc/group/yizhanglab/yiz/venv/stvit-cpu/lib64/python3.9/site-packages/anndata/utils.py:260: UserWarning: Suffix used (-[0-9]+) to deduplicate index values may make index values difficult to interpret. There values with a similar suffixes in the index. Consider using a different delimiter by passing `join={delimiter}`Example key collisions generated by the make_index_unique algorithm: ['SNORD116-1', 'SNORD116-2', 'SNORD116-3', 'SNORD116-5', 'SNORD116-6']
  warnings.warn(


### QC and Mitochondrial control: we use the thresholds from observations

In [9]:

adata.var['mt'] = adata.var_names.str.startswith('MT-') | adata.var_names.str.startswith('mt-')
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

MIN_GENES = 100
MIN_COUNTS = 100
MIN_CELLS = 3
random_state = 42

sc.pp.filter_cells(adata, min_genes = MIN_GENES)  
sc.pp.filter_cells(adata, min_counts = MIN_COUNTS )   
sc.pp.filter_genes(adata, min_cells = MIN_CELLS)

MAX_MITO_PERCENT=27.77

print('pre mt filtering ', adata.shape[0])
adata = adata[adata.obs.pct_counts_mt < MAX_MITO_PERCENT , :]
print('post mt filtering ', adata.shape[0])


pre mt filtering  370115
post mt filtering  303350


In [10]:
adata.obs['n_counts'].values.mean()

12301.415025548047

In [ ]:
#sc.pl.tsne(adata,color='group', save='crc98_ref_celltype_umap.pdf')



# Step 2: Convert gene counts into normalized gene expression per-cell type for gene selection
- proportion of genes sum to 1 for a cell

In [23]:
import STHD.refscrna

In [30]:
from STHD.refscrna import gene_lambda_by_ct
genemeanpd, genemeanpdfc = gene_lambda_by_ct(adata, ctcol = 'group')

[Log] 300 cells for cell type cM03 (DC1)
dimension of cell by gene count (300, 31873)
[Log] 11362 cells for cell type cB2 (B GC-like)
dimension of cell by gene count (11362, 31873)
[Log] 722 cells for cell type cS05 (Endo venous)
dimension of cell by gene count (722, 31873)
[Log] 15824 cells for cell type cP2 (Plasma IgG)
dimension of cell by gene count (15824, 31873)
[Log] 485 cells for cell type cTNI12 (CD8+ IL7R+)
dimension of cell by gene count (485, 31873)
[Log] 386 cells for cell type cS10 (Endo tip cells)
dimension of cell by gene count (386, 31873)
[Log] 94 cells for cell type cM08 (AS-DC)
dimension of cell by gene count (94, 31873)
[Log] 1299 cells for cell type Tumor cE05 (Enterocyte 2)
dimension of cell by gene count (1299, 31873)
[Log] 518 cells for cell type cS21 (Fibro stem cell niche)
dimension of cell by gene count (518, 31873)
[Log] 788 cells for cell type cTNI22 (cTNI22)
dimension of cell by gene count (788, 31873)
[Log] 4516 cells for cell type cTNI11 (CD8+GZMK+)
dim

# Step 3: Select some cell type-informative genes.

- Min gene lambda 0.000125 , logFC 0.5. 
- This is example cutoff refers to RCTD STYLE: "Using the estimated cell type expression profiles 𝜇ˆ𝑘,𝑗, we select differentially expressed genes that will be informative when estimating cell type proportions. For each cell type in the scRNA-seq reference, we select genes with minimum average expression above .0625 counts per 500 and at least 0.5 log-fold-change compared to the average expression across all cell types. "

In [36]:
from STHD.refscrna import select_ct_informative_genes_basic
genemeanpd_filtered = select_ct_informative_genes_basic(genemeanpd, genemeanpdfc, exprcut = 0.000125, fccut = 2**0.5)

31734 genes after removing MT- and Ribo
4618 genes after further removing RCTD threshold gene expression 0.0125
4618 genes after further removing RCTD logfc log2 0.5
(4618, 98)


In [37]:
# check a few gene markers to see whether they are in.
colon_epithelial_marker = ['LEFTY1','EPHB3','LGR5','ALDH1B1','CDCA7','OLFM4','STMN1','HMGB2','MKI67','CLCA4','AQP8','TMIGD1','GUCA2B','SULT1A2','SLC26A3','SLC26A2','CA1',
           'SPINK1','CLCA1','ITLN1','LRRC26','RETNLB','SPINK4','FCGBP','MUC2','ZG16','BCAS1','ENTPD8','MUC1','TFF1','S100P','NOTCH2NL','OTOP3','REN','BEST4','OTOP2','CA7',
          'AZGP1','HCK','LRMP','SH2D6','HPGDS','PTGS1','CHGA','CRYBA2','SCGN','PYY','SCT','INSL5'] 
','.join([t for t in colon_epithelial_marker  if t in genemeanpd_filtered.index])

'LEFTY1,ALDH1B1,CDCA7,OLFM4,STMN1,HMGB2,MKI67,CLCA4,AQP8,TMIGD1,GUCA2B,SULT1A2,SLC26A3,SLC26A2,CA1,SPINK1,CLCA1,ITLN1,LRRC26,RETNLB,SPINK4,FCGBP,MUC2,ZG16,BCAS1,ENTPD8,MUC1,TFF1,S100P,NOTCH2NL,BEST4,OTOP2,CA7,AZGP1,HCK,LRMP,SH2D6,HPGDS,PTGS1,CHGA,CRYBA2,SCGN,PYY,SCT,INSL5'

## Now, Re-calculate the gene labmdas for the selected celltype-informative genes


In [38]:
adata_markergs = adata[:, genemeanpd_filtered.index]
genemeanpd_markergs, genemeanpdfc_markergs = gene_lambda_by_ct(adata_markergs, ctcol = 'group')

[Log] 300 cells for cell type cM03 (DC1)
dimension of cell by gene count (300, 4618)
[Log] 11362 cells for cell type cB2 (B GC-like)
dimension of cell by gene count (11362, 4618)
[Log] 722 cells for cell type cS05 (Endo venous)
dimension of cell by gene count (722, 4618)
[Log] 15824 cells for cell type cP2 (Plasma IgG)
dimension of cell by gene count (15824, 4618)
[Log] 485 cells for cell type cTNI12 (CD8+ IL7R+)
dimension of cell by gene count (485, 4618)
[Log] 386 cells for cell type cS10 (Endo tip cells)
dimension of cell by gene count (386, 4618)
[Log] 94 cells for cell type cM08 (AS-DC)
dimension of cell by gene count (94, 4618)
[Log] 1299 cells for cell type Tumor cE05 (Enterocyte 2)
dimension of cell by gene count (1299, 4618)
[Log] 518 cells for cell type cS21 (Fibro stem cell niche)
dimension of cell by gene count (518, 4618)
[Log] 788 cells for cell type cTNI22 (cTNI22)
dimension of cell by gene count (788, 4618)
[Log] 4516 cells for cell type cTNI11 (CD8+GZMK+)
dimension of 

# Step 3: Save the gene expression lambdas

In [40]:
#outfile = '../gene_lambda_file.txt'
genemeanpd_markergs.to_csv(outfile,sep='\t')


In [39]:
genemeanpd_markergs

,cM03 (DC1),cB2 (B GC-like),cS05 (Endo venous),cP2 (Plasma IgG),cTNI12 (CD8+ IL7R+),cS10 (Endo tip cells),cM08 (AS-DC),Tumor cE05 (Enterocyte 2),cS21 (Fibro stem cell niche),cTNI22 (cTNI22),...,cS22 (Fibro stem cell niche),cTNI20 (PLZF+ T),Tumor cE01 (Stem/TA-like),cTNI15 (CD8+ CXCL13+ HSP+),Tumor cE04 (Enterocyte 1),cE04 (Enterocyte 1),cTNI18 (gd-like T PDCD1+),cB1 (B IGD+IgM+),cE01 (Stem/TA-like),cP3 (Plasma IgG prolif)
SAMD11,0.000002,1.354575e-07,0.000008,1.201868e-06,0.000000e+00,2.414196e-07,0.000004,0.000000e+00,1.471183e-05,0.000000,...,0.000039,3.665324e-07,5.086971e-07,0.000000e+00,0.000000e+00,2.969258e-08,7.679331e-07,1.571570e-07,9.711975e-07,4.587755e-07
HES4,0.000013,1.165777e-05,0.000024,4.646142e-06,3.429627e-05,9.395580e-05,0.000052,1.226572e-04,1.098259e-05,0.000009,...,0.000016,9.994381e-05,6.711329e-05,1.021592e-05,1.102458e-04,4.320694e-05,1.901902e-05,9.532449e-06,3.761882e-05,6.681818e-06
ISG15,0.000371,2.396111e-04,0.000769,1.925330e-04,2.311966e-04,4.572697e-04,0.000529,4.971079e-04,1.773285e-04,0.000452,...,0.000202,6.084152e-04,6.048460e-04,8.492946e-04,7.696574e-03,6.667285e-03,7.051271e-04,2.246783e-04,8.444898e-05,1.927991e-04
TNFRSF18,0.000006,8.850565e-06,0.000007,2.501304e-05,1.415558e-04,1.726362e-05,0.000011,2.664117e-06,1.609288e-07,0.000084,...,0.000002,1.472609e-03,2.307296e-06,5.548547e-04,5.822551e-07,1.339242e-06,5.439643e-04,3.157481e-06,1.783881e-06,2.542230e-05
TNFRSF4,0.000006,6.559940e-06,0.000058,1.902282e-05,3.706697e-05,9.563515e-04,0.000051,2.876440e-06,2.534074e-06,0.000062,...,0.000002,5.529597e-05,2.146784e-06,1.335309e-04,2.988338e-06,1.819685e-07,3.784740e-05,2.829275e-06,1.436634e-06,1.265869e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATP6AP1,0.000092,4.578629e-05,0.000107,3.338679e-05,8.206867e-05,1.854192e-04,0.000216,8.972633e-05,8.903323e-05,0.000043,...,0.000094,6.446808e-05,1.308182e-04,8.931441e-05,1.345284e-04,7.459446e-05,5.469417e-05,4.041530e-05,8.255739e-05,5.271606e-05
FAM50A,0.000057,6.297009e-05,0.000098,2.755123e-05,1.120739e-04,1.255818e-04,0.000125,1.129033e-04,1.115468e-04,0.000110,...,0.000091,1.541112e-04,1.449286e-04,1.354185e-04,9.213218e-05,5.820652e-05,1.227087e-04,5.150738e-05,9.278696e-05,5.626052e-05
LAGE3,0.000081,9.172346e-05,0.000126,4.414699e-05,4.329798e-05,1.111914e-04,0.000143,2.785378e-05,4.156548e-05,0.000099,...,0.000049,1.705586e-04,1.806062e-04,8.740751e-05,3.099945e-05,1.530810e-05,1.374337e-04,8.629313e-05,1.143672e-04,1.138378e-04
F8,0.000001,7.350558e-06,0.000035,6.585625e-07,2.725360e-06,6.688461e-06,0.000000,3.451038e-08,6.042331e-06,0.000012,...,0.000004,6.544941e-06,2.584643e-06,1.148462e-05,3.076564e-06,2.969258e-08,1.202523e-05,8.105909e-07,2.945273e-06,1.505940e-06
